In [1]:
import pymongo
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017/')
db = client['tmldb']

In [2]:
from datetime import datetime

In [3]:
collection = db['instruments']

In [4]:
from bson.son import SON
fut_chains = collection.aggregate(
[
   {
      '$lookup':
        {
          "from": "contracts",
          "localField": 'idinstrument',
          "foreignField": 'idinstrument',
          "as": "futures_chains"
        },
   },
   {   
       "$sort": SON([("futures_chains.expirationdate", -1)])
   }
   
   
])

In [5]:
db.options.create_index([('idcontract', pymongo.ASCENDING),('expirationdate', pymongo.ASCENDING), ('strike', pymongo.ASCENDING),  ])

'idcontract_1_expirationdate_1_strike_1'

In [6]:
import time
start = time.time()

def get_opt_chain(idcontract):
    c = db.options.aggregate([
    {'$match': { 'idcontract' : idcontract }},     
    {'$sort': SON([('expirationdate', 1), ('strikeprice', 1)])},
    {'$group': { "_id" : "$expirationdate",
                "strikes": { "$addToSet": "$strikeprice" }}},

    ])

    expirations_values = []
    for item in c:
        c2 = db.options.aggregate([
            {'$match': { 'idcontract' : idcontract }},     
            {'$match': { 'expirationdate' : item['_id'] }},
            {'$match': { 'strikeprice' : {"$in": item['strikes']} }},     
            {'$group': { "_id" : "$strikeprice",
                        'chain': { "$push": "$$ROOT" }}},
            {'$sort': SON([('_id', 1)])},   ])

        strikes_values = []
        for s in c2:
            #print(s)
            strikes_values.append(s)
            #pprint.pprint(s)
        item['strikes'] = strikes_values

        #pprint.pprint(item)
        expirations_values.append(item)
    
    return expirations_values

import pprint
for instrument in  collection.find():
    fut_chains = []
    for contract in db.contracts.find({'idinstrument': instrument['idinstrument']}).sort('expirationdate'):
        option_chains = []
        
        contract['option_chains'] = get_opt_chain(contract['idcontract'])        
        #pprint.pprint(contract)
        fut_chains.append(contract)
        
    instrument['fut_chains'] = fut_chains
    
    #pprint.pprint(instrument)
    #instrument['_id'] = instrument['cqgsymbol']
    del instrument['_id']
    db.assetindex.insert_one(instrument)
    

    
    
end = time.time()
print(end - start)

KeyboardInterrupt: 

In [ ]:
for option in db.options.find({'idcontract': 7079}).sort([('expirationdate', 1)]):
    print(option)

In [28]:
db.options.create_index([('exchangesymbol', pymongo.ASCENDING)])
    

'exchangesymbol_1'

In [20]:
sym = db.instruments.find({'exchangesymbol':'CL'}).next()

In [21]:
db.futures_data.find({'exchangesymbol':'CL'}).next()

{'_id': ObjectId('577a4f7c4b01f47f84ca9ca4'),
 'admcode': 'CU',
 'admexchangecode': '07',
 'admfuturepricefactor': 1.0,
 'admoptionftpfilestrikedisplay': 50.0,
 'admoptionpricefactor': 1.0,
 'commissionpercontract': 2.99,
 'cqgsymbol': 'CLE',
 'customdayboundarytime': datetime.datetime(1900, 1, 1, 11, 15),
 'datastart': datetime.datetime(2009, 1, 1, 0, 0),
 'decisionoffsetminutes': 5,
 'description': 'Crude Oil',
 'enabled': 1,
 'exchangesymbol': 'CL',
 'exchangesymbolTT': 'CL',
 'idAssetClass': 3,
 'idexchange': 3,
 'idinstrument': 21,
 'idinstrumentgroup': 4,
 'instrumentid_eod': 0,
 'instrumentsymbol_pre_eod': '',
 'instrumentsymboleod_eod': '',
 'limittickoffset': 10,
 'listedspread': 0,
 'margin': 4510,
 'modeled': 1,
 'notes': '',
 'optionadmstrikedisplay': 50.0,
 'optiondatamonthscollected': 12,
 'optionenabled': 2,
 'optionexchangesymbol': 'LO',
 'optionexchangesymbolTT': 'LO',
 'optionstart': datetime.datetime(2008, 1, 1, 0, 0),
 'optionstrikedisplay': 50.0,
 'optionstrikedisp

In [8]:
import json
import pickle
import gzip

instr_name = 'EP'
sym = db.instruments.find({'symbol':instr_name}).next()


with gzip.GzipFile(instr_name+'_instrument.pgz', 'w') as f:
    pickle.dump(sym, f)

In [9]:
with gzip.GzipFile(instr_name+'_instrument.pgz', 'r') as f:
    chain_data = pickle.load(f)

In [10]:
sym

{'_id': ObjectId('579211d991254451abe6c975'),
 'admcode': 'ES',
 'admexchangecode': '16',
 'admfuturepricefactor': 1.0,
 'admoptionftpfilestrikedisplay': 5.0,
 'admoptionpricefactor': 1.0,
 'commissionpercontract': 2.0,
 'cqgsymbol': 'EP',
 'customdayboundarytime': datetime.datetime(1900, 1, 1, 12, 50),
 'datastart': datetime.datetime(2009, 1, 1, 0, 0),
 'decisionoffsetminutes': 5,
 'description': 'S&P Mini',
 'enabled': 1,
 'exchangesymbol': 'ES',
 'exchangesymbolTT': 'ES',
 'idAssetClass': 4,
 'idexchange': 2,
 'idinstrument': 11,
 'idinstrumentgroup': 5,
 'instrumentid_eod': 73,
 'instrumentsymbol_pre_eod': 'EP',
 'instrumentsymboleod_eod': 'SP',
 'limittickoffset': 10,
 'listedspread': 0,
 'margin': 3850,
 'modeled': 1,
 'notes': '',
 'optionadmstrikedisplay': 5.0,
 'optiondatamonthscollected': 36,
 'optionenabled': 2,
 'optionexchangesymbol': 'ES',
 'optionexchangesymbolTT': 'ES',
 'optionstart': datetime.datetime(1900, 1, 1, 0, 0),
 'optionstrikedisplay': 50.0,
 'optionstrikedisp

In [11]:
import json
import pickle
import gzip

fut_chains = []
for contract in db.contracts.find({
        'idinstrument': sym['idinstrument'], 
        'expirationdate': {'$gt': datetime(2014, 1, 1, 0,0,0)}}).sort('expirationdate').limit(12):    
    fut_chains.append(contract)


with gzip.GzipFile(instr_name+'_futures.pgz', 'w') as f:
#with open('CL.data', 'bw') as f:
    pickle.dump(fut_chains, f)

In [12]:
fut_chains

[{'_id': ObjectId('579211e591254451abe6da53'),
  'contractname': 'F.US.EPH14',
  'cqgsymbol': 'F.EPH14',
  'expirationdate': datetime.datetime(2014, 3, 21, 0, 0),
  'idcontract': 4736,
  'idinstrument': 11,
  'month': 'H',
  'monthint': 3,
  'year': 2014},
 {'_id': ObjectId('579211e591254451abe6da55'),
  'contractname': 'F.US.EPM14',
  'cqgsymbol': 'F.EPM14',
  'expirationdate': datetime.datetime(2014, 6, 20, 0, 0),
  'idcontract': 4738,
  'idinstrument': 11,
  'month': 'M',
  'monthint': 6,
  'year': 2014},
 {'_id': ObjectId('579211e591254451abe6da5c'),
  'contractname': 'F.US.EPU14',
  'cqgsymbol': 'F.EPU14',
  'expirationdate': datetime.datetime(2014, 9, 19, 0, 0),
  'idcontract': 4745,
  'idinstrument': 11,
  'month': 'U',
  'monthint': 9,
  'year': 2014},
 {'_id': ObjectId('579211e591254451abe6da7c'),
  'contractname': 'F.US.EPZ14',
  'cqgsymbol': 'F.EPZ14',
  'expirationdate': datetime.datetime(2014, 12, 19, 0, 0),
  'idcontract': 4777,
  'idinstrument': 11,
  'month': 'Z',
  'mo

In [106]:
[x['expirationdate'] for x in fut_chains]

[datetime.datetime(2014, 3, 21, 0, 0),
 datetime.datetime(2014, 6, 20, 0, 0),
 datetime.datetime(2014, 9, 19, 0, 0),
 datetime.datetime(2014, 12, 19, 0, 0),
 datetime.datetime(2015, 3, 20, 0, 0),
 datetime.datetime(2015, 6, 19, 0, 0),
 datetime.datetime(2015, 9, 18, 0, 0),
 datetime.datetime(2015, 12, 18, 0, 0),
 datetime.datetime(2016, 3, 18, 0, 0),
 datetime.datetime(2016, 6, 17, 0, 0),
 datetime.datetime(2016, 9, 16, 0, 0),
 datetime.datetime(2016, 12, 16, 0, 0)]

In [101]:
fut_ids = []

for f in fut_chains:
    fut_ids.append(f['idcontract'])

In [102]:
fut_ids

[4736, 4738, 4745, 4777, 4803, 5022, 5161, 5265, 5546, 5700, 5951, 6570]

In [13]:
import json
import pickle
import gzip

opt_chains = []
for contract in db.options.aggregate([
    {'$match': {'idcontract': 4736}},
    {'$sort': { 'strikeprice': 1} },    
    {'$group': {
        '_id': {'date': '$expirationdate'},
        'chain': {'$push': '$$ROOT'},
        }
    },    
    {'$sort':{"_id.date":1}}   
    ]):
    opt_chains.append(contract)

with gzip.GzipFile(instr_name+'_options.pgz', 'w') as f:
#with open('CL.data', 'bw') as f:
    pickle.dump(opt_chains, f)

In [14]:
opt_chains

[{'_id': {'date': datetime.datetime(2014, 1, 17, 0, 0)},
  'chain': [{'_id': ObjectId('57920fc091254451abe1b8cb'),
    'callorput': 'C',
    'cqgsymbol': 'C.US.EPF145500',
    'expirationdate': datetime.datetime(2014, 1, 17, 0, 0),
    'idcontract': 4736,
    'idinstrument': 11,
    'idoption': 11488724,
    'optionmonth': 'F',
    'optionmonthint': 1,
    'optionname': 'C.US.EPF145500',
    'optionyear': 2014,
    'strikeprice': 550.0},
   {'_id': ObjectId('57920fc091254451abe1b96d'),
    'callorput': 'P',
    'cqgsymbol': 'P.US.EPF145500',
    'expirationdate': datetime.datetime(2014, 1, 17, 0, 0),
    'idcontract': 4736,
    'idinstrument': 11,
    'idoption': 11488886,
    'optionmonth': 'F',
    'optionmonthint': 1,
    'optionname': 'P.US.EPF145500',
    'optionyear': 2014,
    'strikeprice': 550.0},
   {'_id': ObjectId('57920fc091254451abe1b8cc'),
    'callorput': 'C',
    'cqgsymbol': 'C.US.EPF145750',
    'expirationdate': datetime.datetime(2014, 1, 17, 0, 0),
    'idcontract'

In [107]:
d = {}

In [108]:
d['sd']

KeyError: 'sd'

In [11]:
db.futures_data.find({'datetime': datetime(2014, 1, 6, 10,15, 0), 'idcontract': 4736})

In [ ]:
db.futures_data.create_index([('idcontract', pymongo.ASCENDING),('datetime', pymongo.ASCENDING)])